In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///family.db

In [ ]:
%%sql

select first_name
from person, child_of
where mother_id = id
and original_surname = new_surname
and (birth_date > 50);

In [ ]:
%%sql

-- without subquery - use join

SELECT first_name
FROM person
JOIN child_of ON person.id = child_of.mother_id
WHERE person.original_surname = person.new_surname
AND (strftime('%Y', 'now') - strftime('%Y', birth_date)) > 50;


In [ ]:
%%sql

-- again without join

select first_name
from person
where original_surname = new_surname
and (strftime('%Y', 'now') - strftime('%Y', birth_date)) > 50
and id in
    (select mother_id
    from child_of);

In [ ]:
%%sql
-- sqllite dont support!!
select count(child_id)
from child_of
group by (father_id, mother_id)

In [ ]:
%%sql

SELECT father_id, mother_id, COUNT(child_id) as num_children
FROM child_of
GROUP BY father_id, mother_id;

In [ ]:
%%sql

-- now for maried couples only:

select m_id, w_id, count(child_id) as num_children
from  marriage, child_of
group by mother_id, father_id
having m_id = father_id and w_id = mother_id


When you are trying to perform a query on multiple tables using an implicit JOIN (by listing the tables separated by commas), you'll need to specify the conditions for how the tables relate to each other in the WHERE clause. This is known as an implicit join condition.

Here's the corrected query:

```sql

%%sql

SELECT m.m_id, m.w_id, COUNT(c.child_id) as num_children
FROM marriage m, child_of c
WHERE m.m_id = c.father_id AND m.w_id = c.mother_id
GROUP BY m.m_id, m.w_id;
```

Changes made:

    Added table aliases for clarity: m for marriage and c for child_of.
    Moved the conditions specifying the relationship between the marriage and child_of tables to the WHERE clause.

The resulting query should give you the count of children for each married couple.

In [ ]:
%%sql

SELECT m.m_id, m.w_id, COUNT(c.child_id) as num_children
FROM marriage m, child_of c
WHERE m.m_id = c.father_id AND m.w_id = c.mother_id
GROUP BY m.m_id, m.w_id;

11
12
3
5

In [ ]:
%%sql
-- lets creat 3 generation's family name
insert into person values 
    ('11', 'Levs grndfather','Ayzen', 'Ayzen', 1900-01-01),
    ('111', 'Levs grandmother', 'Ayzen', 'Ayzen', 1901-10-10),
    ('12', 'Yossi', 'Ayzen', 'Ayzen', 1929-10-10),
    ('122', 'Levs mom','Ayzen', 'Ayzen', 1929-10-10)
    ;

insert into child_of values
    ('12', '11', '111' ), -- yosi
    ('3', '12','122'); -- lev

insert into marriage values
    ('11', '111', '1920-01-01'), --grabdoarabts
    ('12', '122', '1960-01-01'); -- parants

In [ ]:
%%sql

select original_surname as old_surname
from person, child_of
where id = child_id
group by original_surname
having original_surname not exists in
    select original_surname
    from person, child_of
    where id = child_id
    group by original_surname
    not in
        select original_surname
        from person, child_of
        where id = child_id;

In [ ]:
%%sql

SELECT p1.original_surname AS old_surname
FROM person p1
WHERE EXISTS ( 
    SELECT 1 FROM child_of c1 WHERE c1.child_id = p1.id 
    AND EXISTS ( 
        SELECT 1 FROM child_of c2 WHERE (c2.child_id = c1.father_id OR c2.child_id = c1.mother_id)
        AND EXISTS (
            SELECT 1 FROM child_of c3 WHERE (c3.child_id = c2.father_id OR c3.child_id = c2.mother_id)
        )
    )
)
GROUP BY p1.original_surname;